MediaGamma - Take Home Test
======

Qikai Gu

July 17, 2018


The test is about building a CTR prediction model with one of provided datasets. 
I picked [https://www.kaggle.com/c/avazu-ctr-prediction](https://www.kaggle.com/c/avazu-ctr-prediction).

It's expected to:
- finish the test in 2-3 hours
- return the results in 2 weeks but the sooner the better
- build an end-to-end pipeline for the task
- showcase my understanding of various aspects of ML: ETL, model building and selection, evaluation, etc.
- develop in python (jupyter notebooks) with reasonable comments
- use version control with appropriate commit messages

In [1]:
import numpy as np
import pandas as pd
import xlearn as xl
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Use only 'train.gz' here as we need labeled data to evaluate our models.
df = pd.read_csv('input/train.gz', # nrows=100000,
                 compression='gzip', dtype={'id': str}, index_col='id')

In [3]:
pd.set_option('display.max_columns', 100)
df.head()

,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
id,,,,,,,,,,,,,,,,,,,,,,,
1000009418151094273,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,ddd2926e,44956a24,1,2,15706,320,50,1722,0,35,-1,79
10000169349117863715,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,96809ac8,711ee120,1,0,15704,320,50,1722,0,35,100084,79
10000371904215119486,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,b3cf8def,8a4875bd,1,0,15704,320,50,1722,0,35,100084,79
10000640724480838376,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,e8275b8f,6332421a,1,0,15706,320,50,1722,0,35,100084,79
10000679056417042096,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,07d7df22,a99f214a,9644d0bf,779d90c2,1,0,18993,320,50,2161,0,35,-1,157


In [4]:
print('dataset shape: \t{}'.format(df.shape))

dataset shape: 	(40428967, 23)


In [5]:
# Check class balance
df.click.value_counts()

0    33563901
1     6865066
Name: click, dtype: int64

In [6]:
# Show features and types

pd.set_option('display.max_rows', 30)
df.dtypes

click                int64
hour                 int64
C1                   int64
banner_pos           int64
site_id             object
site_domain         object
site_category       object
app_id              object
app_domain          object
app_category        object
device_id           object
device_ip           object
device_model        object
device_type          int64
device_conn_type     int64
C14                  int64
C15                  int64
C16                  int64
C17                  int64
C18                  int64
C19                  int64
C20                  int64
C21                  int64
dtype: object

In [7]:
# Check if there's any NaN values
for col in df.columns:
    print('{:20}: {}'.format(col, df[col].hasnans))

click               : False
hour                : False
C1                  : False
banner_pos          : False
site_id             : False
site_domain         : False
site_category       : False
app_id              : False
app_domain          : False
app_category        : False
device_id           : False
device_ip           : False
device_model        : False
device_type         : False
device_conn_type    : False
C14                 : False
C15                 : False
C16                 : False
C17                 : False
C18                 : False
C19                 : False
C20                 : False
C21                 : False


In [8]:
pd.set_option('display.max_rows', 15)

# Print value counts about site
for col in ['site_id', 'site_domain', 'site_category']:
    print(df[col].value_counts())

85f751fd    14596137
1fbe01fe     6486150
e151e245     2637747
d9750ee7      963745
5b08c53b      913325
5b4d2eda      771360
856e6d3f      765891
              ...   
25147cbf           1
7d220afa           1
415259a0           1
8bfe3df6           1
0dc19f16           1
0854eadf           1
f793fd01           1
Name: site_id, Length: 4737, dtype: int64
c4e18dd6    15131739
f3845767     6486150
7e091613     3325008
7687a86e     1290165
98572c79      996816
16a36ef3      855686
58a89a43      765891
              ...   
b5bd3e30           1
5816c7e5           1
aba25695           1
09e25a99           1
e1540884           1
34dca79e           1
3123141a           1
Name: site_domain, Length: 7745, dtype: int64
50e219e0    16537234
f028772b    12657073
28905ebd     7377208
3e814130     3050306
f66779e6      252451
75fa27f6      160985
335d28a8      136463
              ...   
9ccfa2ea         318
c706e647          28
da34532e          23
74073276          14
110ab22d           6
6432c423 

In [9]:
# Print value counts about app
for col in ['app_id', 'app_domain', 'app_category']:
    print(df[col].value_counts())

ecad2386    25832830
92f5800b     1555283
e2fcccd2     1129016
febd1138      759098
9c13b419      757812
7358e05e      615635
a5184c22      491457
              ...   
3b997c04           1
79f28324           1
41a5f5f4           1
7656d44f           1
8d6c6b56           1
1be9fcc5           1
2ae0203c           1
Name: app_id, Length: 8552, dtype: int64
7801e8d9    27237087
2347f47a     5240885
ae637522     1881838
5c5a694b     1129228
82e27996      759125
d9b5648e      713924
0e8616ad      660510
              ...   
cab5b73d           1
e1f23257           1
1dc4224a           1
61b00f7b           1
3d459804           1
4e33d472           1
aa396318           1
Name: app_domain, Length: 559, dtype: int64
07d7df22    26165592
0f2161f8     9561058
cef3e649     1731545
8ded1f7a     1467257
f95efa07     1141673
d1327cf5      123233
09481d60       54886
              ...   
71af18ce           5
86c1a5a3           3
6fea3693           2
ef03ae90           2
f395a87f           1
cba0e20d    

In [10]:
# Print value counts about device
for col in ['device_id', 'device_ip', 'device_model']:
    print(df[col].value_counts())

a99f214a    33358308
0f7c61dc       21356
c357dbff       19667
936e92fb       13712
afeffc18        9654
987552d1        4187
28dc8687        4101
              ...   
0bcc0190           1
61424339           1
47530fa4           1
9a37d452           1
e3cc30f1           1
d3dfe1ca           1
abbcf2c5           1
Name: device_id, Length: 2686408, dtype: int64
6b9769f2    208701
431b3174    135322
2f323f36     88499
af9205f9     87844
930ec31d     86996
af62faf4     85802
009a7861     85382
             ...  
7245676b         1
f5aae2e7         1
6d3607ad         1
4bee6fde         1
6fce7808         1
c73be097         1
36e62e5f         1
Name: device_ip, Length: 6729486, dtype: int64
8a4875bd    2455470
1f0bc64f    1424546
d787e91b    1405169
76dc4769     767961
be6db1d7     742913
a0f5f879     652751
4ea23a13     645153
             ...   
eb7d45a5          1
799f9351          1
9eee7d14          1
e29c91f9          1
a2a6267e          1
d40d5789          1
0da00937          1
Name: 

In [11]:
# Print some more information about device
for col in ['device_type', 'device_conn_type']:
    print(df[col].value_counts())

1    37304667
0     2220812
4      774272
5      129185
2          31
Name: device_type, dtype: int64
0    34886838
2     3317443
3     2181796
5       42890
Name: device_conn_type, dtype: int64


In [28]:
# There are many rows having device_id = 'a99f214a', tricky
# TODO
df.loc[df.device_id == 'a99f214a'].device_ip.value_counts()

6b9769f2    182360
431b3174    117691
2f323f36     88134
af9205f9     87402
930ec31d     86576
af62faf4     85387
009a7861     84986
             ...  
d752f769         1
9da21250         1
4d2c6413         1
a2d1d2f6         1
9ac946d3         1
150844a8         1
0586dfb4         1
Name: device_ip, Length: 5389452, dtype: int64

In [36]:
vc = df.device_id.value_counts()
vc['0f7c61dc']

21356

In [26]:
# df.loc[df.site_id == '85f751fd']

In [27]:
# df.loc[df.app_id == 'ecad2386']

In [17]:
# Define a class to structure field and feature mappings

class FFMap(dict):
    """A fields and features map structured as:
    {
        'fields': {
            field1: {
                'idx': field1_idx,
                'features': {
                    feature1: feature1_idx,
                    ...
                }
                'feature_cnt': 1
            },
            ...
        },
        'field_cnt': 0
    }
    """
    def __init__(self):
        super(FFMap, self).__init__(
            {'fields': {}, 
             'field_cnt': 0})
    
    def _has_field(self, field):
        return field in self['fields']
    
    def _has_feature(self, field, feature):
        return self._has_field and \
               feature in self['fields'][field]['features']
    
    def _add_feature(self, field, feature):
        self['fields'][field]['features'][feature] = \
            self['fields'][field]['feature_cnt']
        self['fields'][field]['feature_cnt'] += 1
    
    def _get_field_idx(self, field):
        return self['fields'][field]['idx']
    
    def _get_feature_idx(self, field, feature):
        return self['fields'][field]['features'][feature]
    
    def _get_value(self, field, feature):
        return '{}:{}:1'.format(self._get_field_idx(field), 
                                self._get_feature_idx(field, feature))
        
    def update_field(self, field):
        if self._has_field(field):
            return

        self['fields'][field] = {
            'idx': self['field_cnt'],
            'features': {},
            'feature_cnt': 0
        }
        self['field_cnt'] += 1
        
    def convert_feature(self, field, feature):
        if not self._has_feature(field, feature):
            self._add_feature(field, feature)
        return self._get_value(field, feature)

In [18]:
def convert_feature(ffmap, field, feature):
    return ffmap.convert_feature(field, feature)

def df_to_ffm(df, ffmap, fields, type_, label_col=None):
    ffm = pd.DataFrame()
    if label_col:
        ffm[label_col] = df[label_col]
    
    for field in fields:
        ffmap.update_field(field)
        ffm[field] = df[field].apply(
            lambda feature: convert_feature(ffmap, field, feature))
    
    ffm.to_csv('{}.ffm'.format(type_), sep=' ', header=False, index=False)

In [37]:
!rm *.ffm
!rm *.ffm.bin

In [20]:
%%time

ffmap = FFMap()
df_to_ffm(df[:60000], ffmap=ffmap, fields=df.columns[1:], type_='train', label_col='click')
df_to_ffm(df[60000:80000], ffmap=ffmap, fields=df.columns[1:], type_='valid', label_col='click')
df_to_ffm(df[80000:100000], ffmap=ffmap, fields=df.columns[1:], type_='test')

CPU times: user 3.6 s, sys: 52 ms, total: 3.65 s
Wall time: 3.65 s


In [21]:
%%time

ffm_model = xl.create_ffm()
ffm_model.setTrain('train.ffm')
ffm_model.setValidate('valid.ffm')

params = {'task': 'binary', 
          'k': 2,
          'lr': 0.03,
          'lambda': 1e-3,
          'metric': 'auc',
          'epoch': 1000
         }

ffm_model.fit(params, 'model.out')

CPU times: user 32.2 s, sys: 64 ms, total: 32.2 s
Wall time: 3.38 s


In [22]:
ffm_model.setTest('test.ffm')
ffm_model.setSigmoid()
ffm_model.predict('model.out', 'output.txt')

In [23]:
# Evaluation
y_true = df[80000:100000]['click'].values
with open('output.txt', 'r') as f:
    y_pred = np.array([float(p) for p in f.readlines()])

In [ ]:
fpr, tpr, _ = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)
roc_auc

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC - Click Trough Rate')
plt.legend(loc="lower right")
plt.show()